In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from functions import mean_gen, varx_gen, vary_gen, spherical_data, test_on_integers, learn_parameters, likelihood_ratio, compare_learning
from tensorflow.python.framework.ops import disable_eager_execution

2022-08-22 13:39:01.029591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-22 13:39:01.029630: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
# Random Seed
rand_n = 12345

# Single Point on Sphere
n = 30000
angles = np.array([1.53, 1.67])
thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)


# Likelihood Ratio of Data
l_out = likelihood_ratio(x_train)

In [16]:
# LIKELIHOOD MODEL

loss_fn0 = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold0 = tf.keras.Input(shape=(1,))
simple_linear0 = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold0)
model_angles0 = Model(inputs = inputs_hold0, outputs = simple_linear0)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs0 = tf.keras.Input(shape=(2,))
inputs0 = tf.keras.layers.concatenate([raw_inputs0, model_angles0(tf.ones_like(raw_inputs0)[:,0:1])])
output0 = likelihood_ratio(inputs0)

model_paramfinder = Model(inputs = raw_inputs0, outputs = output0)
model_paramfinder.compile(loss=loss_fn0, optimizer='Adam')
# model_paramfinder.summary()

In [17]:
# INTERPOLATED MODEL

model_interpolate = tf.keras.models.load_model('3dmodels/discrete_model_mth50_mph50')

for l in model_interpolate.layers:
    l.trainable=False
    
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold = tf.keras.Input(shape=(1,))
simple_linear = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs = tf.keras.Input(shape=(2,))
inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
output = model_interpolate(inputs)

model_interpolate_paramfinder = Model(inputs = raw_inputs, outputs = output)
model_interpolate_paramfinder.compile(loss=loss_fn, optimizer='Adam')
# model_interpolate_paramfinder.summary()



x_inputs = x_train[:, 0:2]

In [18]:
epochs, iterations, batch_size = 1, 50, 300

# # Likelihood Stats
# weights0 = learn_parameters(model_paramfinder, x_train, y_train, iterations, epochs, batch_size)
# angle_errors0 = (angles - weights0) / angles
# average_error0 = np.sum(np.abs(angle_errors0)) / 2


# Comparing
weights1, weights2 = compare_learning(model_paramfinder, model_interpolate_paramfinder, angles,
                                      x_train, y_train, iterations, epochs, batch_size)
# angle_errors = (angles - weights) / angles
# average_error = np.sum(np.abs(angle_errors)) / 2

# print('Average likelihood error %:', average_error0 * 100)
# print('Average interpolated error %:', average_error * 100)
# print('')

160/160 [==============================] - 1s 3ms/step - loss: 0.6453
After  0 iterations, LE %:  28.371548652648926  IE %:  29.278206825256348
160/160 [==============================] - 0s 3ms/step - loss: 0.6413
After  1 iterations, LE %:  21.35595828294754  IE %:  23.56596291065216
160/160 [==============================] - 1s 3ms/step - loss: 0.6384
After  2 iterations, LE %:  16.224604845046997  IE %:  19.476184248924255
160/160 [==============================] - 1s 3ms/step - loss: 0.6374
After  3 iterations, LE %:  12.464720010757446  IE %:  16.450494527816772
160/160 [==============================] - 1s 3ms/step - loss: 0.6371
After  4 iterations, LE %:  9.651985764503479  IE %:  14.22756016254425
160/160 [==============================] - 1s 4ms/step - loss: 0.6367
After  5 iterations, LE %:  7.48085081577301  IE %:  12.619057297706604
160/160 [==============================] - 1s 3ms/step - loss: 0.6366
After  6 iterations, LE %:  5.798245593905449  IE %:  11.449721455574036

160/160 [==============================] - 1s 3ms/step - loss: 0.6364
After  38 iterations, LE %:  0.2763212891295552  IE %:  8.864161372184753
160/160 [==============================] - 1s 3ms/step - loss: 0.6365
After  39 iterations, LE %:  0.28734314255416393  IE %:  8.814655244350433
160/160 [==============================] - 1s 3ms/step - loss: 0.6363
After  40 iterations, LE %:  0.20225723274052143  IE %:  8.889670670032501
160/160 [==============================] - 1s 3ms/step - loss: 0.6365
After  41 iterations, LE %:  0.37448741495609283  IE %:  8.967860043048859
160/160 [==============================] - 1s 3ms/step - loss: 0.6366
After  42 iterations, LE %:  0.23253117688000202  IE %:  9.105166792869568
160/160 [==============================] - 1s 3ms/step - loss: 0.6362
After  43 iterations, LE %:  0.25469502434134483  IE %:  8.82413387298584
160/160 [==============================] - 1s 3ms/step - loss: 0.6365
After  44 iterations, LE %:  0.30263676308095455  IE %:  9.004

In [33]:
# MANUAL TESTING

# xr = model_interpolate(x_train).numpy().transpose()[0]
# y = model_interpolate(x_train)
# loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
# print(loss)

# x_train_mod = np.copy(x_train)
# x_train_mod[:, 2] = x_train[:, 2]
# x_train_mod[:, 3] = x_train[:, 3] - .1
# xr_mod = model_interpolate(x_train_mod).numpy().transpose()[0]
# y_mod = model_interpolate(x_train_mod)
# loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
# print(loss_mod)

# xr = likelihood_ratio(x_train).numpy().transpose()
# y = likelihood_ratio(x_train)
# loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
# print(loss)

# x_train_mod = np.copy(x_train)
# x_train_mod[:, 2] = x_train[:, 2] - .05
# x_train_mod[:, 3] = x_train[:, 3] - .05
# xr_mod = likelihood_ratio(x_train_mod).numpy().transpose()
# y_mod = likelihood_ratio(x_train_mod)
# loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
# print(loss_mod)

0.6019409317687167
